In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.discrete.discrete_model import Logit
from sklearn.metrics import roc_auc_score

In [2]:
# LDL, Height, BMI, diabetes, renal failure, heart attack, asthma, non-melanoma skin cancer
# pheno_list = ['INI50','INI21001','HC221','HC294','HC326','HC382','cancer1060']
pheno_list=['BIN_FC1002247']
phenos = pd.read_table('/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.phe',
                       usecols=['IID','age','sex','PC1','PC2','PC3','PC4']+pheno_list, 
                       index_col=['IID'],
                       na_values=-9)
phenos[[i for i in phenos.columns if 'INI' not in i]] -= 1
phenos.loc[~phenos.index.duplicated(keep='first')]
phenos.describe()

,age,sex,PC1,PC2,PC3,PC4,BIN_FC1002247
count,488377.000000,488377.000000,488377.000000,488377.000000,488377.000000,488377.000000,479457.000000
mean,64.454925,-0.542335,-2.123313,-0.640132,-1.089341,-0.870598,0.266737
std,8.112788,0.498205,54.305102,27.858077,14.891779,10.403127,0.442254
min,45.000000,-1.000000,-20.269900,-283.317000,-144.258000,-108.573000,0.000000
25%,58.000000,-1.000000,-14.335200,1.445080,-3.673770,-1.910279,0.000000
50%,66.000000,-1.000000,-13.161900,2.648770,-2.506000,0.210960,0.000000
75%,71.000000,0.000000,-11.773500,3.762990,-1.248864,2.575510,1.000000
max,82.000000,0.000000,418.396000,85.112400,97.822600,36.103100,1.000000


In [3]:
a,b = 'white_british', 'non_british_white'
with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+a+'.phe', 'r') as f:
    pop = {int(line.split()[0]) for line in f}

with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+b+'.phe', 'r') as f:
    pop2 = {int(line.split()[0]) for line in f}

In [ ]:
z='all_beta_center_p001_20190530_500PCs'
npz = np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/all_pop/tsvd/'+z+'.npz')
scores = pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/scorefiles/'+z+'_full.profile',
                       sep='\s+',
                       index_col='IID')
scores = scores.loc[scores.index.isin(phenos.index),:]
phenos = phenos.loc[phenos.index.isin(scores.index),:]

scores.iloc[:,:10].head()

In [ ]:
for p,name in zip([pop,pop2],['White British:','Non-British White:']):
    print(name)
    for phe in pheno_list:
        print(phe),
        pcvec = npz['V'][np.where(npz['label_phe_code'] == phe),:].flatten()
        score = scores.loc[scores.index.isin(p),:].sort_index().dot(pcvec).dropna()
        if 'INI' in phe:
            pheno = ols(phe+'~age+sex+PC1+PC2+PC3+PC4',
                        data=phenos.loc[phenos.index.isin(p),:]).fit().resid.sort_index().dropna()
            pheno = pheno.subtract(pheno.mean()).divide(pheno.std())
        else:
            pheno = phenos.loc[phenos.index.isin(p),phe].sort_index().dropna()
        data = pd.concat([score,pheno], axis=1)
        if 'INI' in phe:
            print('r^2={:.4}'.format(data.corr(method='spearman').iloc[0,1]**2))
        else:
            print('AUC={:.4}'.format(roc_auc_score(data.iloc[:,1].astype(bool), data.iloc[:,0])))
    print('')